In [1]:
from pyspark.sql import SparkSession
import os
import sys

In [2]:
# get DB credentials from environment variables
DATABASE_USER_NAME = 'projetooto'
DATABASE_PASSWORD = 'teste2022@2023'

In [3]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4a57cf83-c442-4166-a716-da07cbe538df;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [4]:
bucket_name = 'hering-datalake-prod-southamerica-east1-bucket-silver-zone'
db_table_names = 'CLIENTES_ABORDAVEIS_CRM'
db_host = '10.20.6.136'
db_port = '1433'
db_name = 'DB_ANALISE'

print("*****************")
print("Dados recebidos:")
print(bucket_name)
print(db_table_names)
print(db_host)
print(db_port)
print(db_name)
print("*****************")

server_name = f"jdbc:sqlserver://{db_host}:{db_port}"
url = server_name + ";" + "databaseName=" + db_name + ";"

*****************
Dados recebidos:
hering-datalake-prod-southamerica-east1-bucket-silver-zone
CLIENTES_ABORDAVEIS_CRM
10.20.6.136
1433
DB_ANALISE
*****************


In [5]:
def extract(table_name: str) :
    df_table = (
        spark.read
            .format("com.microsoft.sqlserver.jdbc.spark")
            .option("url", url)
            .option("query", f"SELECT * FROM {table_name} WHERE MONTH(DATA_REFERENCIA) = MONTH(GETDATE()) + 1 AND YEAR(DATA_REFERENCIA) = YEAR(GETDATE())")
            .option("user", DATABASE_USER_NAME)
            .option("password", DATABASE_PASSWORD)
            .load()
    )
    return df_table

In [9]:
def load(df_table):        
    (
        df_table.write 
            .mode("overwrite")
            .format("delta")
            .save(f"gs://{bucket_name}/oto/bnsvher361/db_analise/dbo/clientes_abordaveis_crm/")
    )

In [10]:
df_table = extract('CLIENTES_ABORDAVEIS_CRM')

In [11]:
load(df_table)

In [8]:
    for table_name in db_table_names.split(':::'):
        try:
            print("*****************")
            print(f"Carregando {table_name}")
            print("*****************")

            df_table = extract(table_name)
            
            print("*****************")
            print(f"Escrevendo {table_name}")
            print("*****************")
            
            load(df_table)
            
            print("*****************")
            print(f"Tabela {table_name} concluida!")
            print("*****************")
        #FIXME: set exceptions type
        except Exception as error :
            print(f"Falha na tabela {table_name}!!!")
            print(error)

*****************
Carregando CLIENTES_ABORDAVEIS_CRM
*****************
*****************
Escrevendo CLIENTES_ABORDAVEIS_CRM
*****************


Falha na tabela CLIENTES_ABORDAVEIS_CRM!!!
An error occurred while calling o55.save.
: java.lang.RuntimeException: java.lang.NoSuchMethodException: com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem.<init>(java.net.URI, org.apache.hadoop.conf.Configuration)
	at org.apache.hadoop.fs.AbstractFileSystem.newInstance(AbstractFileSystem.java:144)
	at org.apache.hadoop.fs.AbstractFileSystem.createFileSystem(AbstractFileSystem.java:173)
	at org.apache.hadoop.fs.AbstractFileSystem.get(AbstractFileSystem.java:258)
	at org.apache.hadoop.fs.FileContext$2.run(FileContext.java:336)
	at org.apache.hadoop.fs.FileContext$2.run(FileContext.java:333)
	at java.base/java.security.AccessController.doPrivileged(Native Method)
	at java.base/javax.security.auth.Subject.doAs(Subject.java:423)
	at org.apache.hadoop.security.UserGroupInformation.doAs(UserGroupInformation.java:1730)
	at org.apache.hadoop.fs.FileContext.getAbstractFileSystem(FileContext.java:333)
	at org.apache.hadoop.fs.FileContext.getFileConte

In [9]:
    spark.stop()